## Table des matières

- [Présentation des données](#Intro)
- [1. Correction des données de consommation mensuelles de l'effet température (dues au chauffage électrique) en utilisant une régression linéaire](#chapitre_1)
- [2. Désaisonnalisation de la consommation obtenue après correction, grâce aux moyennes mobiles](#chapitre_2)
- [3. Prévision de la consommation (corrigée de l'effet température) sur un an, en utilisant la méthode de Holt Winters (lissage exponentiel) puis la méthode SARIMA sur la série temportelle](#chapitre_3)

In [1]:
import numpy as np
import pandas as pd

<a id="Intro"></a>
## <span style="color: #030aa4">Présentation des données</span>

Les données mensuelles de consommation d'électricité en énergie proviennent du site https://www.rte-france.com/eco2mix/telecharger-les-indicateurs

In [19]:
DonneesMensuelles = pd.read_csv("eCO2mix_RTE_energie_M.csv", sep='\t')

DonneesMensuelles.head(3)

,Mois,Qualite,Territoire,Production totale,Production nucleaire,Production thermique totale,Production thermique charbon,Production thermique fioul,Production thermique gaz,Production hydraulique,...,Production bioenergies,Consommation totale,Solde exportateur,Echanges export,Echanges import,Echanges avec le Royaume-Uni,Echanges avec l'Espagne,Echanges avec l'Italie,Echanges avec la Suisse,Echanges avec l'Allemagne et la Belgique
0,2012-01,Donnees definitives,France,57177,42811.0,5399,741.0,691,3967,6741,...,514,51086,5371.0,7043.0,1672.0,929.0,724.0,1581.0,2195.0,-59.0
1,2012-02,Donnees definitives,France,54419,38661.0,8721,2511.0,1309,4901,5244,...,454,54476,-715.0,4504.0,5220.0,-798.0,-309.0,1244.0,1579.0,-2430.0
2,2012-03,Donnees definitives,France,48583,37549.0,5276,1435.0,666,3175,3894,...,531,43156,4888.0,7042.0,2153.0,583.0,330.0,1742.0,2235.0,-1.0


In [16]:
DonneesMensuelles.columns

Index(['Mois', 'Qualite', 'Territoire', 'Production totale',
       'Production nucleaire', 'Production thermique totale',
       'Production thermique charbon', 'Production thermique fioul',
       'Production thermique gaz', 'Production hydraulique',
       'Production eolien', 'Production solaire', 'Production bioenergies',
       'Consommation totale', 'Solde exportateur', 'Echanges export',
       'Echanges import', 'Echanges avec le Royaume-Uni',
       'Echanges avec l'Espagne', 'Echanges avec l'Italie',
       'Echanges avec la Suisse', 'Echanges avec l'Allemagne et la Belgique'],
      dtype='object')

Le dataframe se compose de 22 colonnes :
- Mois : indique le mois analysé
- Qualite : trois types de données : temps réel, consolidées et définitives. Données temps réel disponibles 30 jours, puis remplacées par données consolidées. Données consolidées remplacées par donées définitives l'année suivante
- Territoire : indique le territoire concerné. Choix entre la France et une des 12 régions administratives.
- Production totale :  production totale d'électricité en MW
- Production nucléaire/thermique totale/thermique charbon/thermique fioul/thermique gaz/hydraulique/eolien/solaire/bioenergies : indique la quantité de production d'électricité produite en MW selon la source de production 
- Consommation totale : consommation totale d'électricité en MW
- Solde exportateur : somme des quantités d'électricité en MW importées et exportées
- Echanges exports : quantité d'électricité en MW exportée
- Echanges imports : quantité d'électricité en MW importée
- Echanges avec R-U/Espagne/Italie/Suisse/Allemagne et Belgique : détails des échanges avec chaque pays indiqué

Source des informations : https://assets.rte-france.com/prod/public/2020-07/%C3%A9CO2mix%20-%20Description%20des%20fichiers%20des%20donn%C3%A9es%20en%20puissance.pdf

Les données de l'effet température proviennent du site https://cegibat.grdf.fr/simulateur/calcul-dju 

In [24]:
Dju_chauffage = pd.read_csv('dju_chauffage.csv')

Dju_chauffage.head(3)

,Unnamed: 0,JAN,FÉV,MAR,AVR,MAI,JUN,JUI,AOÛ,SEP,OCT,NOV,DÉC,TOTAL
0,2010,624.8,474.7,414.9,292.7,260.6,112.6,46.5,93.8,163.6,286.4,419.3,644.1,3833.8
1,2011,507.2,408.6,368.7,202.0,165.6,117.2,122.5,84.7,101.9,232.3,338.0,424.8,3073.4
2,2012,454.5,555.6,334.9,344.4,192.8,133.3,95.3,59.6,165.5,262.1,403.3,455.7,3456.9


Le jeu de données 'Dju_chauffage' est composée de 14 colonnes et 9 lignes. 12 de ces colonnes indiquent le degré jour unifié pour chaque mois de l'année. la première colonne nous indique l'année concernée et la dernière le total annuel. Chaque ligne correspond à une année pour la période allant de 2010 à 2018.

Le degré jour est une valeur représentative de l’écart entre la température d’une journée donnée et un seuil de température préétabli (18 °C dans le cas des DJU ou Degré Jour Unifié). Sommés sur une période, ils permettent de calculer les besoins de chauffage d’un bâtiment.
    
Source : https://cegibat.grdf.fr/simulateur/calcul-dju

<a id="chapitre_1"></a>
## <span style="color: #030aa4">1. Correction des données de consommation mensuelles de l'effet température (dues au chauffage électrique) en utilisant une régression linéaire</span>

In [33]:
data = DonneesMensuelles.loc[:,('Mois','Territoire','Consommation totale')]
data = data[data.Territoire=='France']
data = data.rename(columns={'Consommation totale':'consommation_totale_france'})\
           .drop(columns='Territoire')

data.head(3)

,Mois,consommation_totale_france
0,2012-01,51086
1,2012-02,54476
2,2012-03,43156


In [ ]:
Dju_clim = Dju_clim.pivot(index ='Produit', columns='Unnamed:0', values='')


In [12]:
Dju_clim = pd.read_csv('dju_clim.csv')

<a id="chapitre_2"></a>
## <span style="color: #030aa4">2. Désaisonnalisation de la consommation obtenue après correction, grâce aux moyennes mobiles</span>

<a id="chapitre_3"></a>
## <span style="color: #030aa4">3. Prévision de la consommation (corrigée de l'effet température) sur un an, en utilisant la méthode de Holt Winters (lissage exponentiel) puis la méthode SARIMA sur la série temportelle</span>